# Emojifier V2: Using LSTMs in Keras

In [51]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import emoji
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(1)

In [52]:
train_df = pd.read_csv('data/train_emoji.csv', header=None, names=['phrase','emoji'], usecols=[0,1])
test_df = pd.read_csv('data/test_emoji.csv', header=None, names=['phrase','emoji'], usecols=[0,1])

In [53]:
train_df.head()

,phrase,emoji
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [54]:
X_train = train_df['phrase'].to_numpy()
Y_train = train_df['emoji'].to_numpy()

X_test = test_df['phrase'].to_numpy()
Y_test = test_df['emoji'].to_numpy()

In [55]:
maxLen = len(max(X_train, key=len).split())

In [56]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

In [57]:
index = 1
print(X_train[index], label_to_emoji(Y_train[index]))

I am proud of your achievements 😄


In [58]:
 Y_oh_train = pd.get_dummies(Y_train).to_numpy()
 Y_oh_test = pd.get_dummies(Y_test).to_numpy()

In [59]:
index = 50
print(Y_train[index], "is converted into one hot", Y_oh_train[index])

0 is converted into one hot [1 0 0 0 0]


In [60]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [61]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

In [62]:
print("Total words in the voucabulary: ", len(word_to_index))
print("Word vector dim: ", word_to_vec_map[index_to_word[1]].shape[0])

Total words in the voucabulary:  400000
Word vector dim:  50


In [63]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [64]:
index = 289846
print("Embending vector for word ", index_to_word[index])
word_to_vec_map[index_to_word[index]]

Embending vector for word  potatos


array([-0.77033 , -0.57765 , -1.4086  ,  0.28488 ,  1.3224  ,  0.025708,
       -0.39524 ,  0.1565  ,  0.73926 ,  0.07319 ,  0.39573 ,  0.81557 ,
        1.7829  , -1.1323  , -1.2555  , -0.31784 , -1.0604  , -0.19973 ,
        2.2133  , -0.13883 ,  0.50197 ,  1.5084  ,  0.58036 ,  0.3317  ,
        0.56073 ,  2.767   ,  0.22387 , -1.1203  ,  1.3909  ,  0.51904 ,
        0.91258 ,  1.2222  ,  1.3356  , -0.01816 ,  1.855   ,  0.87767 ,
       -0.51949 , -0.60569 , -0.33336 , -0.68257 ,  0.18867 , -1.2253  ,
       -0.3839  , -0.61717 ,  1.311   ,  1.0898  ,  0.23086 ,  1.3371  ,
       -0.060975,  1.4675  ])

Convert all your training sentences into lists of indices, and then zero-pad all these lists so that their length is the length of the longest sentence.

In [65]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """

    m = X.shape[0]      # number of training examples

    X_indices = np.zeros((m, max_len))

    for i in range(m):
        sentence_words = X[i].lower().split()

        j = 0

        for w in sentence_words:
            X_indices[i,j] = word_to_index[w]
            j = j + 1
    
    return X_indices

In [66]:

X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [67]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))         # (400001, 50)
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [68]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

In [69]:
print("embedding layer input  dim: ", embedding_layer.input_dim)
print("embedding layer output dim: ", embedding_layer.output_dim)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])
print("index od word 'apple': ", word_to_index["apple"])
print("word at index 57797: ", index_to_word[57797])
print("vec map of word 'apple': ", word_to_vec_map["apple"])
print("embending vec of word at index 57797: ", embedding_layer.get_weights()[0][57797])


embedding layer input  dim:  400001
embedding layer output dim:  50
weights[0][1][3] = -0.3403
index od word 'apple':  57797
word at index 57797:  apple
vec map of word 'apple':  [ 0.52042  -0.8314    0.49961   1.2893    0.1151    0.057521 -1.3753
 -0.97313   0.18346   0.47672  -0.15112   0.35532   0.25912  -0.77857
  0.52181   0.47695  -1.4251    0.858     0.59821  -1.0903    0.33574
 -0.60891   0.41742   0.21569  -0.07417  -0.5822   -0.4502    0.17253
  0.16448  -0.38413   2.3283   -0.66682  -0.58181   0.74389   0.095015
 -0.47865  -0.84591   0.38704   0.23693  -1.5523    0.64802  -0.16521
 -1.4719   -0.16224   0.79857   0.97391   0.40027  -0.21912  -0.30938
  0.26581 ]
embending vec of word at index 57797:  [ 0.52042  -0.8314    0.49961   1.2893    0.1151    0.057521 -1.3753
 -0.97313   0.18346   0.47672  -0.15112   0.35532   0.25912  -0.77857
  0.52181   0.47695  -1.4251    0.858     0.59821  -1.0903    0.33574
 -0.60891   0.41742   0.21569  -0.07417  -0.5822   -0.4502    0.17253
 

## Building the Emojifier-V2

![emojifier v2](images/emojifier-v2.png)

In [70]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(5, activation=None)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=[sentence_indices], outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [71]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645 

In [72]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:

X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)

In [75]:

model.fit(X_train_indices, Y_oh_train, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
5/5 [==============================] - 20s 24ms/step - loss: 1.6089 - accuracy: 0.2511
Epoch 2/50
5/5 [==============================] - 0s 22ms/step - loss: 1.5168 - accuracy: 0.3213
Epoch 3/50
5/5 [==============================] - 0s 22ms/step - loss: 1.4365 - accuracy: 0.3482
Epoch 4/50
5/5 [==============================] - 0s 21ms/step - loss: 1.3787 - accuracy: 0.3956
Epoch 5/50
5/5 [==============================] - 0s 22ms/step - loss: 1.2804 - accuracy: 0.5275
Epoch 6/50
5/5 [==============================] - 0s 23ms/step - loss: 1.0835 - accuracy: 0.6716
Epoch 7/50
5/5 [==============================] - 0s 23ms/step - loss: 0.9928 - accuracy: 0.6774
Epoch 8/50
5/5 [==============================] - 0s 21ms/step - loss: 0.8956 - accuracy: 0.6761
Epoch 9/50
5/5 [==============================] - 0s 22ms/step - loss: 0.7721 - accuracy: 0.7403
Epoch 10/50
5/5 [==============================] - 0s 20ms/step - loss: 0.5714 - accuracy: 0.8052
Epoch 11/50
5/5 [===========

In [76]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
loss, acc = model.evaluate(X_test_indices, Y_oh_test)
print()
print("Test accuracy = ", acc)

2/2 [==============================] - 1s 7ms/step - loss: 0.3935 - accuracy: 0.8571

Test accuracy =  0.8571428656578064


In [77]:
# This code allows you to see the mislabelled examples
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: he got a very nice raise	❤️
Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:❤️ prediction: I love taking breaks	😞
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😄 prediction: she said yes	😞
Expected emoji:😄 prediction: What you did was awesome	😞
Expected emoji:😞 prediction: go away	⚾
